In [1]:
from googleapiclient.discovery import build
import pandas as pd
import os

In [2]:
api_key = os.environ.get('API_KEY')

In [3]:
if api_key:
    print("API key loaded successfully.")
else:
    print("Failed to load API key.")

API key loaded successfully.


In [4]:
api_service_name = "youtube"
api_version = "v3"
channel_id = "UCoOae5nYA7VqaXzerajD0lg"

# Get Video ID for all videos of a playlist 

https://www.youtube.com/watch?v=T6hmdrsLQj8&list=PL7BImOT2srcGtOFPLa_cPC_FBE8OXkIoz

In [8]:
def get_playlist_videos(playlist_id):
    """
    Fetches video IDs from a given YouTube playlist.

    Parameters:
    playlist_id (str): The ID of the YouTube playlist.

    Returns:
    list: A list of video IDs in the playlist.
    """
    youtube = build(api_service_name, api_version, developerKey=api_key)
    
    video_ids = []
    next_page_token = None

    # Iterate through all pages of the playlist
    while True:
        # Request a page of videos
        pl_request = youtube.playlistItems().list(
            part='contentDetails',
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )
        pl_response = pl_request.execute()
        
        # Extract video IDs from the response and add them to the list
        video_ids += [item['contentDetails']['videoId'] for item in pl_response['items']]
        
        next_page_token = pl_response.get('nextPageToken')
        if not next_page_token:
            break
    
    return video_ids

def get_video_comments(video_id):
    """
    Fetches comments from a given YouTube video.

    Parameters:
    video_id (str): The ID of the YouTube video.

    Returns:
    list: A list of comments from the video.
    """
    youtube = build(api_service_name, api_version, developerKey=api_key)
    
    comments = []
    next_page_token = None

    # Iterate through all pages of comments
    while True:
        # Request a page of comments
        comment_request = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            maxResults=100,
            pageToken=next_page_token
        )
        comment_response = comment_request.execute()
        
        # Extract comments from the response and add them to the list
        for item in comment_response['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)
        
        next_page_token = comment_response.get('nextPageToken')
        if not next_page_token:
            break
    
    return comments

def get_video_title(video_id):
    """
    Fetches the title of a given YouTube video.

    Parameters:
    video_id (str): The ID of the YouTube video.

    Returns:
    str: The title of the video.
    """
    youtube = build(api_service_name, api_version, developerKey=api_key)
    request = youtube.videos().list(
        part="snippet",
        id=video_id
    )
    response = request.execute()

    # Extract the title from the response
    if response['items']:
        return response['items'][0]['snippet']['title']
    else:
        return None

def main():
    """
    Main function to fetch comments and titles from videos in a YouTube playlist.

    Returns:
    DataFrame: A pandas DataFrame containing video IDs, titles, and comments.
    """
    playlist_id = 'PL7BImOT2srcGtOFPLa_cPC_FBE8OXkIoz'  # Replace with your playlist ID
    video_ids = get_playlist_videos(playlist_id)
    # video_titles = get_video_title(video_id)

    all_comments = []
    # Iterate through each video ID, fetch title and comments, and store in a list
    for video_id in video_ids:
        title = get_video_title(video_id)
        comments = get_video_comments(video_id)
        all_comments.append({'video_id': video_id, 'title': title, 'comments': comments})

    # Convert the list of comments to a DataFrame
    df = pd.DataFrame(all_comments)
    return df

# Run the main function
df = main()


In [9]:
df

,video_id,title,comments
0,T6hmdrsLQj8,How to Build Your Ultimate Productivity System,"[How to remember the system then?, 15/3/24, x2..."
1,FbSNfj2S6Pw,5 Essential Tips for Long-Lasting Productivity,"[Nice video, most people remain poor, cos frie..."
2,Bry8a_7b9aM,"How I Organize My Life - The ""LAZI"" Productivi...",[Get free access to my Workflow and Deep Dive ...
3,tQSKyvjsUuI,My Complete Productivity System,[📚Pre-order my book to get an exclusive ticket...
4,iONDebHX9qk,How I Manage My Time - 10 Time Management Tips,[📖 Check out my new book Feel-Good Productivit...
5,RP0oVOH4Zz4,How to Beat Procrastination,"[Thank you sir, Which keyboard are you using....."
6,3LXUVa4ugwc,My Toxic Relationship With Productivity,[Hi Ali 🍀😊 (not sure you’re reading this becau...
7,K-ssUVyfn5g,How to Organise your Life - Building a Second ...,"[Immediately writes down Jerry Seinfeld quote,..."
8,cs7TCNDQy1E,The Most PRODUCTIVE Desk Setups on YouTube | 2021,"[are those white speakers still available?, i ..."
9,qc4GU_wHdqw,12 Productivity Myths Wasting Your Time,[📚Pre-order my book to get an exclusive ticket...


In [7]:
df

,video_id,title,comments
0,T6hmdrsLQj8,How to Build Your Ultimate Productivity System,"[How to remember the system then?, 15/3/24, x2..."
1,FbSNfj2S6Pw,5 Essential Tips for Long-Lasting Productivity,"[Nice video, most people remain poor, cos frie..."
2,Bry8a_7b9aM,"How I Organize My Life - The ""LAZI"" Productivi...",[Get free access to my Workflow and Deep Dive ...
3,tQSKyvjsUuI,My Complete Productivity System,[📚Pre-order my book to get an exclusive ticket...
4,iONDebHX9qk,How I Manage My Time - 10 Time Management Tips,[📖 Check out my new book Feel-Good Productivit...
5,RP0oVOH4Zz4,How to Beat Procrastination,"[Thank you sir, Which keyboard are you using....."
6,3LXUVa4ugwc,My Toxic Relationship With Productivity,[Hi Ali 🍀😊 (not sure you’re reading this becau...
7,K-ssUVyfn5g,How to Organise your Life - Building a Second ...,"[Immediately writes down Jerry Seinfeld quote,..."
8,cs7TCNDQy1E,The Most PRODUCTIVE Desk Setups on YouTube | 2021,"[are those white speakers still available?, i ..."
9,qc4GU_wHdqw,12 Productivity Myths Wasting Your Time,[📚Pre-order my book to get an exclusive ticket...


In [10]:
csv_file = 'youtube_comments.csv'
df.to_csv(csv_file, index=False)

In [11]:
data = pd.read_csv('youtube_comments.csv')

In [12]:
data

,video_id,title,comments
0,T6hmdrsLQj8,How to Build Your Ultimate Productivity System,"['How to remember the system then?', '15/3/24,..."
1,FbSNfj2S6Pw,5 Essential Tips for Long-Lasting Productivity,"['Nice video, most people remain poor, cos fri..."
2,Bry8a_7b9aM,"How I Organize My Life - The ""LAZI"" Productivi...",['Get free access to my Workflow and Deep Dive...
3,tQSKyvjsUuI,My Complete Productivity System,['📚Pre-order my book to get an exclusive ticke...
4,iONDebHX9qk,How I Manage My Time - 10 Time Management Tips,['📖\xa0Check out my new book Feel-Good Product...
5,RP0oVOH4Zz4,How to Beat Procrastination,"['Thank you sir', 'Which keyboard are you usin..."
6,3LXUVa4ugwc,My Toxic Relationship With Productivity,['Hi Ali 🍀😊 (not sure you’re reading this beca...
7,K-ssUVyfn5g,How to Organise your Life - Building a Second ...,['Immediately writes down Jerry Seinfeld quote...
8,cs7TCNDQy1E,The Most PRODUCTIVE Desk Setups on YouTube | 2021,"['are those white speakers still available?', ..."
9,qc4GU_wHdqw,12 Productivity Myths Wasting Your Time,['📚Pre-order my book to get an exclusive ticke...
